# Jedha Fullstack final project
# Title: Interactive Story teller "Horror theme"

This notebook is intended to fine tune an NLP model

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

Work plan:
1. Preparations

# 1) Preparations

#### 1.a) Install the transformers library

First we need to install the transformers library which is a library produced by Hugging-face. It provides APIs and tools to easily download and train pretrained models

In [1]:
#! pip install datasets transformers

#### 1.b) Enable model sharing

To be able to share our model we need to upload it to huggingFace hub "similar to Github".

This gives the chance for the model to be used by any application.

To be able to do so, the  The following 2 instructions are guidelines as per Hugging-face recommendations

##### Hugging face guidelines:


Hugging Face note(1): If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

Hugging Face note(2): To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

Hugging Face guideline: First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password: italicized text

Makesure , when you obtain your authentication token using the link above , to have the "write" permission checked , otherwise you'll face problems in uploading your model, like I did in the first time

Configure your account credentials:

In [2]:
#!pip uninstall accelerate transformers -y
#!pip install accelerate transformers[torch]


In [3]:
#!pip install accelerate -U


In [4]:
#!pip install transformers[torch]


In [5]:
#!git config --global user.email "serdarcekinmez.gmail.com"
#!git config --global user.name "serdarcekinmez"

In [29]:
import os
#os.environ["HUGGINGFACE_TOKEN"] = ""

When you execute the cell below you will receive a prompt to enter your username and password to log in ,
and the token that will enable you to upload your model. if not no worries it will activated automatically

In [30]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [7]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [8]:
import transformers

print(transformers.__version__)

4.30.2


HuggingFace note: You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

# 2) Preparing the Dataset

Mounting the google drive to access the file containing our Horror story dataset

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# importing load_datasets

from datasets import load_dataset


In [11]:
# Hereby is the method:
#datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

# using the horror stories dataset which we exctracted from creepypastas dataset
# using the "Extract-horrotstory" notebook:

datasets = load_dataset("text", data_files={"train": "/content/HorrorC_train.txt", "validation": "/content/HorrorC_test.txt"})

# the following link was used to fine tunuing using draclua story dataset
#datasets = load_dataset("text", data_files={"train": "/content/drive/MyDrive/DataScience/Train/Dracula_train.txt", "validation": "/content/drive/MyDrive/DataScience/Test/Dracula_test.txt"})



  0%|          | 0/2 [00:00<?, ?it/s]

we can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [12]:
datasets["train"][:50]

{'text': ['Six feet and 630 muscular pounds was now average adult size, so at six feet nine inches and 790 pounds with his extra-muscular symbiote J was now considerably taller than average and one of the heaviest and strongest ones. His powerful symbiote had really muscled out for some reason and gained the additional 40 pounds when it formed the extra arms. Everyone had the extra arms regardless of his or her muscular new size, though. J had been richly rewarded for finding a missing diamond wedding ring and returning it to the couple it belonged to three days after being infected and “suited up” by his symbiote and gave most of his reward money to a charity that helped Christian refugees escape unspeakable and horrific persecution in war-torn countries. Many of these people had been rescued with this donation and J’s feeder insect business was thriving. J was able to use the money he earned from selling feeder insects to go diving for tasty lionfish and other prized underwater game.

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [13]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [14]:
show_random_elements(datasets["train"])

,text
0,"Since the beginning of the nuclear age, the threat of mutual destruction has been the only thing keeping the peace between nations…supposedly. It now seemed, however, that the world was evolving into a different place with a different mindset. There were countries in the world with egomaniacal leaders who cared more about power than their citizens and some who were even worse; some that cared only for destruction. There were factions of generational wealthy who funneled ungodly amounts of money into creating propaganda, distrust, riots and eventually…war. To these people war equals money. They just don’t seem to understand that the world has the capacity for only one more war. At the very bottom of the pyramid was the majority of the planet’s seven and a half billion people…and those people were completely terrified; wanting nothing more than peace."
1,"Then, suddenly, I lost my balance, tripped, and fell directly through the wall. I fell face-first onto some dirty tan carpeting. Upon getting back up, I realized that I was in a completely different room. Well, not really a room, per se— more so a set of rooms, all of which connected by openings. The walls were covered in gross tan patterned wallpaper. There was also an overwhelming stench of moist carpet."
2,"I don’t rightly think mice can talk, though."
3,"“Hi, mommy!” Kevin, who just turned six a week ago, smiled and waved at the camera while hovering over the birthday cake which Mike had decorated with stars, planets and a tiny replica of the Olympus Space Station. “I love you,” he continued."
4,“Moreen? Is that you? Are you okay?”
5,"“It’s cool, check out these photos! I think some of them are from Coney Island back in its heyday.”"
6,"The creature crawled up to my chest with agonizing slowness and I kept repeating to myself, You’re mine. You’re mine."
7,"“Who told you you could leave?” he snarled in a sinister voice. Brian looked shocked, until the boorish gentleman smiled and said, “Just kiddin’, little guy.”"
8,
9,"They converted their home world into a paradise (by their standards) and many 10^6s of them poured out into the surrounding system with a rapidity and vigor that we could only envy. With bodies built to survive every environment from the daylit surface of their innermost world, to the atmosphere of their largest gas giant and the cold void in-between, they set out to sculpt their system into something beautiful. At first we thought them simple miners, stripping the rocky planets and moons for vital resources, but then we began to see the purpose to their constructions, the artworks carved into every surface, and traced across the system in glittering lights and dancing fusion trails. And still, our terrible Gift approached."


# 3) Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`distilgpt2`](https://huggingface.co/distilgpt2) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead:

## 3.a Choose the model to fune-tune

We used the Facebook model named opt-350m

OPT stands for : Open Pre-trained Transformer Language Model

**Why did we chose it?**

We chose it for the following three reaons ,
1. It is used for text generation based on prompts
2. It was introduced after the GPT-3 model "*on May 2022*"
3. When testing it it showed very good results



In [15]:
model_checkpoint = "facebook/opt-350m"

## 3.b) Tokenizing

**Why Tokenizing?**

Because in order to deal with texts we need to transfer strings into numbers, the result is a sequnce of integers

**To** tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
# model_Checkpoint here refers to the facebook opt-350m model

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [17]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [18]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [19]:
tokenized_datasets["train"][1]

{'input_ids': [2,
  17,
  48,
  35863,
  6,
  5790,
  6,
  17,
  46,
  5,
  26034,
  8,
  25719,
  1554,
  26,
  6,
  39,
  124,
  7,
  5,
  444,
  2204,
  9,
  5,
  1049,
  929,
  4],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [20]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [21]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [22]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [23]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'iote and gave most of his reward money to a charity that helped Christian refugees escape unspeakable and horrific persecution in war-torn countries. Many of these people had been rescued with this donation and J’s feeder insect business was thriving. J was able to use the money he earned from selling feeder insects to go diving for tasty lionfish and other prized underwater game. With his symbiote he didn’t need a wetsuit, fins, or dive mask for seeing underwater and could stay where the fish he hunted were for much longer. J’s symbiote formed an organic dive mask and'

## 3.c) Training

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. We will a model:

In [24]:
from transformers import AutoModelForCausalLM     # as recommended by Hugging face documentation for opt-350 training
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

And some `TrainingArguments`:

In [25]:
from transformers import Trainer, TrainingArguments
# training arguments facilitates having all the training parameters in one place
# for the ease of modifications as we'll see in the upcoming training command

In [26]:
!pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-HorrorCreepyPasta",  # we chose the name of our model, related to the chosen datast
    evaluation_strategy = "epoch", # the model will be evaluated after each  epoch
    learning_rate=2e-5,   # usually when fine-tunning, low learning rate values are chosen, as per recommendation
    weight_decay=0.01,  # to prevent over-fitting, although here it is a very small value
    push_to_hub=True, # each learning cycle 'epoch'the model is saved in the hub
)

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

We pass along all of those to the `Trainer` class:

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,     # as per the chosen parametrs in the codeing cell above
    train_dataset=lm_datasets["train"],   # the tokenized batched training dataset
    eval_dataset=lm_datasets["validation"], # the tokenized batched validation dataset
)

Cloning https://huggingface.co/CSerdar014191/opt-350m-HorrorCreepyPasta into local empty directory.


And we can train our model:

In [32]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,3.071764
2,No log,3.069607
3,2.963200,3.091719


TrainOutput(global_step=720, training_loss=2.8312357584635417, metrics={'train_runtime': 267.8058, 'train_samples_per_second': 21.452, 'train_steps_per_second': 2.689, 'total_flos': 1338463821496320.0, 'train_loss': 2.8312357584635417, 'epoch': 3.0})

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [33]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 22.01


**What is meant by perplexity?**

it is a measure of how well probability model predicts a sample. To simplify things it means perplexed means confused, consequently the smaller the number means less confusion means better model and vice vers, the bigger the value the worse model is.

Thus, perplexity metric in NLP is a way to capture the degree of 'uncertainty' a model has in predicting (assigning probabilities to) some text.

As we can see here the number seems reasonably well.


You can now **upload the result of the training to the Hub**, just execute this instruction:

In [34]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/1.23G [00:00<?, ?B/s]

Upload file runs/Jun20_10-44-26_715cb2a3693f/events.out.tfevents.1687258355.715cb2a3693f.8632.1:   0%|        …

Upload file runs/Jun20_10-44-26_715cb2a3693f/events.out.tfevents.1687258057.715cb2a3693f.8632.0:   0%|        …

To https://huggingface.co/CSerdar014191/opt-350m-HorrorCreepyPasta
   94c95e4..a2968dd  main -> main

   94c95e4..a2968dd  main -> main

To https://huggingface.co/CSerdar014191/opt-350m-HorrorCreepyPasta
   a2968dd..e3ba866  main -> main

   a2968dd..e3ba866  main -> main



'https://huggingface.co/CSerdar014191/opt-350m-HorrorCreepyPasta/commit/a2968dd72cd87c8d22e6feffd8983db59ddb17d0'

**Important** : Don't forget to upload the tokenizer too, otherwise the model will not work

In [43]:
tokenizer.push_to_hub("CSerdar014191/CreepyPasta-opt-350")

#model_name = model_checkpoint.split("/")[-1]
#training_args = TrainingArguments(
   #f"{model_name}-HorrorCreepyPasta", / for model name find this script above model_checkpioint [-1]= facebook/opt-350 ==> "opt-350"


CommitInfo(commit_url='https://huggingface.co/CSerdar014191/CreepyPasta-opt-350/commit/edf7b18b5390eab9cb1ca69c3986c0276aa76150', commit_message='Upload tokenizer', commit_description='', oid='edf7b18b5390eab9cb1ca69c3986c0276aa76150', pr_url=None, pr_revision=None, pr_num=None)

In [42]:
model.save_pretrained('/content/sample_data')
trainer.push_to_hub(model_name="Fine_tuned_CreepyPasta-opt-350") # this can be found in the read me of hugging Face model file



To https://huggingface.co/CSerdar014191/opt-350m-HorrorCreepyPasta
   e3ba866..1d1deb1  main -> main

   e3ba866..1d1deb1  main -> main



You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sgugger/my-awesome-model")
```

# 4) Generating Text 👍

## 4.1) import our new model

First let's import our fine-tuned model

In [46]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("CSerdar014191/opt-350m-HorrorCreepyPasta")




## 4.2) Import the tokenizer

In [48]:
tokenizer = AutoTokenizer.from_pretrained("CSerdar014191/CreepyPasta-opt-350") #this is tokenizer / not the model on Hugginsface!!



## 4.3) Let's generate a horror Text 😀

In [49]:
prompt = 'I was on a ship'
inputs = tokenizer(prompt, return_tensors="pt")

# the meaning of each parameter is explained in the annex below to avoid taking to much space
outputs = model.generate(**inputs,do_sample=True,min_new_tokens=50,max_new_tokens= 70,temperature = 0.7,top_k=50,top_p=0.95,num_return_sequences=3)

[tokenizer.decode(output) for output in outputs]

['</s>I was on a ship in space, and my crewmate was the god of flight, the one who knew everything about everything. His name was Richard Branson. He had a big fortune, and he had a fleet of ships and a mega-billion-dollar corporation.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '</s>I was on a ship, in the middle of the ocean, when I heard a scream. It was loud, like a scream in a horror movie, but with a different sound. It was a scream of anguish and fear, a scream that was coming from within me. I was afraid it would be my son, and I was afraid that I would die in his wake',
 '</s>I was on a ship for the first time in my life. A ship that I had never seen before. A ship that I didn’t know how to fly. I flew around the ship and watched the ship change colors. The ship changed colors like the sun, but the colors stayed the same. The colors were red, blue, yellow, and green. I']

In [ ]:
[tokenizer.decode(output) for output in outputs]

Look how beautifull the results above are

# 5) Annex

We would like to bring to your kind attention the following points:

1. There is a group of a Jupyter notebook files in which we implemented various types of testings with different:

*   Datasets
*   Models GPT-2 and opt350
*   Text generating parameter settings


2. Hereby, here are the sources used for research

    a. Research for choosing the optimum parameters for text generating:

    https://huggingface.co/blog/introducing-csearch

    b. model documentations: opt-350m
    https://huggingface.co/docs/transformers/main/en/model_doc/opt#transformers.OPTConfig


3. Finally, the definition of the parameters used in generating text command
outputs = model.generate(**inputs,do_sample=True,min_new_tokens=50,max_new_tokens= 70,temperature = 0.7,top_k=50,top_p=0.95,num_return_sequences=3)


**min_new_tokens=50**: This parameter specifies the minimum number of new tokens the generated text should contain. Consequently , it ensures that the generated output is at least the specified length in terms of tokens. This is useful to avoid very short or incomplete responses.

**max_new_tokens=70**: This parameter sets the maximum number of new tokens in the generated text. It limits the length of the output to a specific number of tokens. This can be useful to prevent overly long or verbose responses.

**temperature=0.7**: The temperature parameter controls the randomness of the text generation process. A higher temperature value (greater than 1.0) makes the generated output more diverse and random, while a lower value (less than 1.0) makes it more focused and deterministic. Here, the temperature is set to 0.7, indicating a moderate level of randomness.

**top_k=50**: The top_k parameter specifies the number of top-k words to consider during text generation. It restricts the sampling pool to the most probable k words at each step. A higher value of top_k allows for more diversity in the generated text, while a lower value makes the output more focused and deterministic.
in other words if you choose a low number you will have a poor voacbulary because you have limited the number of words to choose from the corpus of the model and vice-versa

**top_p=0.95**: The top_p, takes the commulative probablity of all probable roots , also known as  "p-coverage" approach, is an alternative to top_k. It considers a cumulative probability distribution and chooses from the smallest set of tokens whose cumulative probability exceeds the given threshold (top_p). A higher value of top_p allows for more diverse and varied outputs, while a lower value makes the output more focused.

**num_return_sequences=3**: This parameter specifies the number of distinct sequences or responses to generate.

Finally a new parameter has been introduced recently , which we used in our project called penalty alpha, it has a beautiful technique that applies a penalty on the words which the model just used, meaning it will not use them until after a while. consequently we can avoid unnecessary repetition and the text becomes more creative






.